In [1]:
import feasible_test as ft
import feasible_test_with_print_error as fterr
import random
import functions as func
import functions_with_print as funcpt
import pandas as pd
import numpy as np

#讀取courseDetail
courseDetail = pd.read_csv('data/course.csv')[['course code', 'Number of students', 'instructor']]
courseDetail['course code']=courseDetail['course code'].astype(str)
RoomDetail = pd.read_csv('data/classroom.csv')[['classroom', 'cr_capacity']]

#variables
roomNum=3 #教室數量
weekdays=5 #上課日子
dailyParts=3 #parts=將一天劃分為[早上、下午、晚上]
period=weekdays*dailyParts #15 一個weekdays中，不分教室的區塊總數
session=roomNum*weekdays*dailyParts #45 一個weekdays中，空教室的總數(一維陣列的長度)
k=weekdays*roomNum #15 [早上、下午、晚上] 一個part中的session數(索引調整參數)
totalCourseNum=30

def get_score(schedule):
    return func.ObjFun(schedule,courseDetail, roomNum, k, RoomDetail, session, period, totalCourseNum)

rule1:  55.24691358024691
[2, 1, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 2, 0]
68 60.0 75.0
rule2:  89.33333333333333
rule3:  30.42424242424242
rule4:  36.0
Objval: 56.320426487093144
56.320426487093144


In [2]:
def cross(schedule_1,schedule_2,cross_index1):
    # select cross index
    cross_index2 = cross_index1 + 6

    # make new schedule
    new_schedule = schedule_1[0: cross_index1] + schedule_2[cross_index1 : cross_index2] + schedule_1[cross_index2 : 45]   
     
    for i in range(cross_index1,cross_index2): #cross範圍
        data = new_schedule[i] # i位置資料 (ex:此處是2)
        data_index = schedule_1.index(data) #此筆資料位於原schedule的什麼位置 (ex:此處2找到位於為schedule "0的位置" )
        new_schedule[data_index] = schedule_1[i] # new_schedule的該位置換成cross對應字串 (ex:此處原"0的位置" 2換成 "i對應位置的6" )

    return new_schedule
  
    

In [3]:
def one_generation(schedule1,schedule2):
    stop_flag= False
    gene_pool = []
    gene_score = []

    for i in range(len(schedule1)-6):
        new_schedule = cross(schedule1,schedule2,i)
        result = ft.feasible_test(new_schedule)    
        if result == True:
            gene_pool.append(new_schedule)
            gene_score.append(float(get_score(new_schedule)))


        new_schedule2 = cross(schedule2,schedule1,i)
        result = ft.feasible_test(new_schedule2)
        if result == True:
            gene_pool.append(new_schedule2)
            gene_score.append(float(get_score(new_schedule)))

    gene_score_sorted = sorted(gene_score, reverse=True)
    next_gen1 = gene_pool[gene_score.index(gene_score_sorted[0])]
    next_gen2 = gene_pool[gene_score.index(gene_score_sorted[1])]
    if max(gene_score_sorted) == min( gene_score_sorted):
        stop_flag = True
    return next_gen1,next_gen2,stop_flag



In [5]:
temp_schedule = ['306000001','','306008001','356461001','','306016002','356425001','','306016012','307873001','','307867001','','','307870001',
 '306016022','307857001','306050011','','356388001','307851001','306525001','','307035001','','356395001','307034001','356564001','306736001','306737001',
'307932001','','356822001','','356389001','356019001','','356387001','307901001','356813001','','356808001','','','307942001']

temp_schedule2 = ['306008001', '307942001', '', '306525001', '356808001', '', '307851001', '306736001', '', '307867001', '307873001', '', '356019001', '356388001', '', '356387001', '356395001', '', '356813001', '306016022', '', '306000001', '307034001', '', '306737001', '356389001', '', '307857001', '306016002', '', '356425001', '307901001', '', '356461001', '307035001', '', '356822001', '307932001', '', '306016012', '356564001', '', '307870001', '306050011', '']

best_score = 0
best_schedule= []
stop_flag = False

print("-----初始解-----")
print(ft.feasible_test(temp_schedule))
if get_score(temp_schedule) > best_score:
    best_score = get_score(temp_schedule)
    best_schedule = temp_schedule
    
print(ft.feasible_test(temp_schedule2))
if get_score(temp_schedule2) > best_score:
    best_score = get_score(temp_schedule2)
    best_schedule = temp_schedule2

next_gen1,next_gen2 = temp_schedule,temp_schedule2

for i in range(1,100):
    print("-----第{}子代-----".format(i))
    print("此子代最高得分為: {}".format(get_score(next_gen1)))
    next_gen1,next_gen2,stop_flag = one_generation(next_gen1,next_gen2)
    if stop_flag:
        break
        
    if get_score(next_gen1) > best_score:
        best_score = get_score(next_gen1)
        best_schedule = next_gen1
    
print("-----已找到最佳解，得分為：{}-----".format(best_score))    
print(best_schedule)

-----初始解-----
True
True
-----第1子代-----
此子代最高得分為: 67.99382716049382
-----第2子代-----
此子代最高得分為: 69.31795735129069
-----第3子代-----
此子代最高得分為: 66.76700336700337
-----已找到最佳解，得分為：73.18383838383838-----
['306008001', '307942001', '', '306525001', '356808001', '', '307851001', '306736001', '', '307867001', '307873001', '', '356019001', '356388001', '', '356387001', '356395001', '', '356813001', '306016022', '', '306000001', '307034001', '', '306737001', '356389001', '', '307857001', '306016002', '', '356425001', '307901001', '', '356461001', '307035001', '', '356822001', '307932001', '', '306016012', '356564001', '', '307870001', '306050011', '']
